## 2 The Data

In [1]:
import wrds
import pandas as pd

In [2]:
# Connect to WRDS
db = wrds.Connection()
db.create_pgpass_file()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
# Query the WRDS Monthly World Indices dataset
data = db.raw_sql("""
    SELECT * 
    FROM wrdsapps_windices.mwcountryreturns
    WHERE date BETWEEN '2002-01-01' AND '2024-12-31'
    AND country IN ('AUSTRIA', 'FRANCE', 'GERMANY', 'JAPAN', 'SWITZERLAND', 'UNITED KINGDOM')
""")
data.to_csv('project_data_MWI.csv', sep=';', index=False)

In [4]:
# Query  CRSP’s value-weighted return index for the US

data = db.raw_sql("""
    SELECT caldt, vwretd, vwretx
    FROM crsp_a_indexes.msp500
    WHERE Caldt BETWEEN '2002-01-01' AND '2024-12-31'""")
data.to_csv('US_stock.csv', sep=';', index=False)

In [5]:
# Query 1-month T-Bill
data = db.raw_sql("""
    SELECT caldt, T30ret
    FROM crsp_a_indexes.mcti 
    WHERE Caldt BETWEEN '2002-01-01' AND '2024-12-31'""")
data.to_csv('US_treasury.csv', sep=';', index=False)